Some resources allow you to do other operations than the classic CRUDS operations. A good example of this are the FHIR Terminology Resources. In this notebook we'll explore **ValueSet/$expand** and **ValueSet/$validate**. 

## ValueSet/$expand

In [21]:
# using the standard Python requests library
# import dependencies
from requests import get, post

# these are some headers that we always want to send with our requests
req_kwargs = {"headers" : {"Accept": "application/fhir+json; fhirVersion=4.3"}}
# let's expand a ValueSet 
vs_expansion = get("http://r4.ontoserver.csiro.au/fhir/ValueSet/$expand?url=http://snomed.info/sct?fhir_vs=isa/363346000&count=10&filter=carcinoma", **req_kwargs).json()
for coding in vs_expansion["expansion"]["contains"]:
    print(coding["code"], coding["display"])

307593001 Carcinomatosis
722688002 Malignant epithelial neoplasm
1156454002 Embryonal carcinoma
449248000 Nasopharyngeal carcinoma
274084007 Palate carcinoma
255037004 Parathyroid carcinoma
254955001 Pituitary carcinoma
269516007 Tongue carcinoma
274085008 Tonsil carcinoma
403929003 Trichilemmal carcinoma


In [ ]:
from fhirpy.base.exceptions import OperationOutcome
from tx_client import SyncFHIRTerminologyClient, r4

# create a ontoserver client
client = SyncFHIRTerminologyClient("http://r4.ontoserver.csiro.au/fhir")

try:
    vs = client.ValueSet().expand(
        url=r4.uri("http://snomed.info/sct?fhir_vs=isa/363346000"),
        filter=r4.string("carcinoma"),
        count=r4.integer(10),
    )
    for coding in vs.expansion.contains:
        print(coding.code, coding.display)
except OperationOutcome as e:
    print(e.resource)

307593001 Carcinomatosis
722688002 Malignant epithelial neoplasm
1156454002 Embryonal carcinoma
449248000 Nasopharyngeal carcinoma
274084007 Palate carcinoma
255037004 Parathyroid carcinoma
254955001 Pituitary carcinoma
269516007 Tongue carcinoma
274085008 Tonsil carcinoma
403929003 Trichilemmal carcinoma


# ValueSet/$validate

In [25]:
from pprint import pprint
# let's expand a ValueSet
validation_result = post("http://r4.ontoserver.csiro.au/fhir/ValueSet/$validate-code",
                        json={
                            "resourceType": "Parameters",
                            "parameter": [
                                {
                                    "name": "url",
                                    "valueUri": "http://snomed.info/sct?fhir_vs=isa/363346000",
                                },
                                {
                                    "name": "coding",
                                    "valueCoding": {
                                        "system": "http://snomed.info/sct",
                                        "code": "363346000",
                                        "display": "carcinoma",
                                    },
                                }
                            ]
                        }, **req_kwargs).json()
pprint(validation_result)

{'parameter': [{'name': 'result', 'valueBoolean': False},
               {'name': 'code', 'valueCode': '363346000'},
               {'name': 'system', 'valueUri': 'http://snomed.info/sct'},
               {'name': 'version',
                'valueString': 'http://snomed.info/sct/32506021000036107/version/20230531'},
               {'name': 'display',
                'valueString': 'Malignant neoplastic disease'},
               {'name': 'message',
                'valueString': 'The provided code '
                               '"http://snomed.info/sct#363346000" exists in '
                               'the ValueSet, but the display "carcinoma" is '
                               'incorrect'},
               {'name': 'issues',
                'resource': {'issue': [{'code': 'invalid',
                                        'details': {'text': 'The provided code '
                                                            '"http://snomed.info/sct#363346000" '
                     

In [27]:
try:
    result = client.ValueSet().validate_code(
        url=r4.uri("http://snomed.info/sct?fhir_vs=isa/363346000"),
        coding=r4.Coding(
            display=r4.string("carcinoma"), 
            code=r4.code("363346000"), 
            system=r4.uri("http://snomed.info/sct")
        ),
    )
    pprint(result)
except OperationOutcome as e:
    pprint(e.resource)

{'code': '363346000',
 'display': 'Malignant neoplastic disease',
 'issues': None,
 'message': 'The provided code "http://snomed.info/sct#363346000" exists in '
            'the ValueSet, but the display "carcinoma" is incorrect',
 'result': False,
 'system': 'http://snomed.info/sct',
 'version': 'http://snomed.info/sct/32506021000036107/version/20230531'}
